# Meetup Project


What is in a Meetup API request?

1. https://api.meetup.com: The API host.
2. /find/groups: The method
3. zip=11211&radius=1&category=25: The args. Here is where you may add the API key.
    From what I can tell, the only args you can give are zip, radius, and category. So maybe you need to go through the zips 
4. &order=members:  How to order the results.


-  Private groups I cannot see the members who go, only the number. We will need to see how many groups are public.


# Imports / Pymongo Setup

In [121]:
from pymongo import MongoClient
import numpy as np
import requests
import json
import warnings
warnings.filterwarnings('ignore')
key = open('api.txt', 'r').read()

client = MongoClient('localhost', 27017)
db = client.meet_up
collection = db.groups

## Get number of groups in each category globally

In [70]:
# This creates a list of dictionaries, with name, sort_name, id, and shortname. 
dict_categories = json.loads(requests.get('https://api.meetup.com/2/categories?key={}'.format(key)).text)['results']

#make list of ids 
ids = [item['id'] for item in dict_categories]
ids.sort()

#define function
def get_count_of_category(number):
    req_count = requests.get('https://api.meetup.com/find/groups?key={}&radius=global&category={}'.format(key, number))\
                        .headers['X-Total-Count']
    cat_name = next(item['name'] for item in dict_categories if item["id"] == number)
    return [number, int(req_count), cat_name]
    
groups_per_category = [get_count_of_category(category) for category in ids]

Total number of groups

In [117]:
sum([val[1] for val in groups_per_category])

344905

I find 344,607 groups globally. Wow!

# Download all Meetup Groups

In [127]:
#Meetup only allows 200 requests at a time.
def get_groups_for_category(number):
    number_of_groups = [entry[1] for entry in groups_per_category if entry[0] == number] 
    offsets = list(range(int(number_of_groups[0]/200) + 1))
    for offset in offsets:
        req = requests.get('https://api.meetup.com/find/groups?key={}\
                            &category={}&radius=global&\
                            &offset={}&page=200'.format(key, number, offset))
        json_data = json.loads(req.text)
        collection.insert_many(json_data)

get_groups_for_category(24)

# Play with the 403 in category 24

First, how many of these are public or private? What categories exist for visibility? Apparently, only 'public' and 'public_limited'. No group in category 24 is private.

I bet that is because, if you are a private group, I can't even download you at all. 


In [141]:
print(collection.distinct('visibility'))

print(collection.find({'visibility': 'public'}).count())

['public', 'public_limited']
299


299/ 403 are public. Good news!

# Download the events for a particular group

In [160]:
req = requests.get('https://api.meetup.com/Florida-MUFON-UFO-Research-and-Investigations/events?\
                    key={}&status=past'.format(key))

print(req.headers['X-Total-Count'])
print(json.loads(req.text)[0])

168
{'created': 1351348658000, 'duration': 10800000, 'id': '88793112', 'name': "Meeting on December 6th, 2012, Mimi's Cafe at the Avenues in Viera.", 'date_in_series_pattern': False, 'status': 'past', 'time': 1354833000000, 'local_date': '2012-12-06', 'local_time': '17:30', 'updated': 1354995467000, 'utc_offset': -18000000, 'waitlist_count': 0, 'yes_rsvp_count': 15, 'venue': {'id': 960032, 'name': "Mimi's Cafe", 'lat': 28.247121810913086, 'lon': -80.72822570800781, 'repinned': False, 'address_1': '6630 Colonnade Avenue', 'city': 'Melbourne', 'country': 'us', 'localized_country_name': 'USA', 'phone': '321-638-0404', 'zip': '32940', 'state': 'FL'}, 'group': {'created': 1351346689000, 'name': 'Florida MUFON UFO Research and Investigations', 'id': 5582872, 'join_mode': 'open', 'lat': 27.969999313354492, 'lon': -80.6500015258789, 'urlname': 'Florida-MUFON-UFO-Research-and-Investigations', 'who': 'UFOlogists', 'localized_location': 'Palm Bay, FL', 'state': 'FL', 'country': 'us', 'region': 'e

# Find the attendees for a particular event

In [178]:
req = requests.get('https://api.meetup.com/Florida-MUFON-UFO-Research-and-Investigations/events/88793112/rsvps?\
                    key={}'.format(key))

#Insert the attendees into their own database.
id_data = json.loads(req.text)
collection = db.attendees
collection.insert_many(id_data)

#get unique identifiers out
collection.distinct('member.id')

[7385260, 26274872, 71777132, 62218232, 16899141, 56597972, 12450926]